In [115]:
import time
import numpy as np

import cplex as CPX
import cplex.callbacks as CPX_CB

import gurobipy as gp
from gurobipy import GRB

import consts
import params

In [116]:
def set_params(c, primal_bound=None,
               branch_strategy=None,
               timelimit=None,
               seed=None,
               test=False):
    if seed is not None:
        c.parameters.randomseed.set(seed)

    # Select from one of the default branching strategies
    if branch_strategy == consts.CPX_DEFAULT:
        c.parameters.mip.strategy.variableselect.set(0)
    elif branch_strategy == consts.CPX_PC:
        c.parameters.mip.strategy.variableselect.set(2)
    elif branch_strategy == consts.CPX_SB:
        c.parameters.mip.strategy.variableselect.set(3)

    # Single threaded
    c.parameters.threads.set(1)

    # Disable primal heuristics
    c.parameters.mip.strategy.heuristiceffort.set(0)

    # Disable presolve
    c.parameters.preprocessing.presolve.set(0)

    c.parameters.mip.tolerances.integrality.set(params.EPSILON)

    c.parameters.mip.strategy.search.set(
        c.parameters.mip.strategy.search.values.traditional)

    # Set the primal bound if provided
    if primal_bound is not None:
        if c.objective.get_sense() == consts.MINIMIZE:
            c.parameters.mip.tolerances.lowercutoff.set(primal_bound)
        else:
            c.parameters.mip.tolerances.uppercutoff.set(primal_bound)

    # Set timelimit if provided
    if timelimit is not None:
        c.parameters.timelimit.set(timelimit)

    # Disable cutting planes
    c.parameters.mip.limits.eachcutlimit.set(0)
    c.parameters.mip.cuts.bqp.set(-1)
    c.parameters.mip.cuts.cliques.set(-1)
    c.parameters.mip.cuts.covers.set(-1)
    c.parameters.mip.cuts.disjunctive.set(-1)
    c.parameters.mip.cuts.flowcovers.set(-1)
    c.parameters.mip.cuts.pathcut.set(-1)
    c.parameters.mip.cuts.gomory.set(-1)
    c.parameters.mip.cuts.gubcovers.set(-1)
    c.parameters.mip.cuts.implied.set(-1)
    c.parameters.mip.cuts.localimplied.set(-1)
    c.parameters.mip.cuts.liftproj.set(-1)
    c.parameters.mip.cuts.mircut.set(-1)
    c.parameters.mip.cuts.mcfcut.set(-1)
    c.parameters.mip.cuts.rlt.set(-1)
    c.parameters.mip.cuts.zerohalfcut.set(-1)

    if not test:
        disable_output(c)


def disable_output(c):
    c.set_log_stream(None)
    c.set_error_stream(None)
    c.set_warning_stream(None)
    c.set_results_stream(None)

In [122]:
def solve_as_lp(c, max_iterations=50):
    disable_output(c)
    # Create LP for the input MIP
    c.set_problem_type(c.problem_type.LP)
    # Set the maximum number of iterations for solving the LP
    if max_iterations is not None:
        c.parameters.simplex.limits.iterations = max_iterations

    c.solve()
    status, objective, dual_values = None, None, None

    status = c.solution.get_status()
    # print(status)
    if status == consts.LP_OPTIMAL or status == consts.LP_ABORT_IT_LIM:
        objective = c.solution.get_objective_value()
        dual_values = c.solution.get_dual_values()

    return status, objective, dual_values


def get_branch_solution(context, cclone, var_idx, bound_type):
    value = context.get_values(var_idx)

    get_bounds = None
    set_bounds = None
    new_bound = None
    if bound_type == consts.LOWER_BOUND:
        get_bounds = context.get_lower_bounds
        set_bounds = cclone.variables.set_lower_bounds
        new_bound = np.floor(value) + 1
    elif bound_type == consts.UPPER_BOUND:
        get_bounds = context.get_upper_bounds
        set_bounds = cclone.variables.set_upper_bounds
        new_bound = np.floor(value)

    original_bound = get_bounds(var_idx)

    set_bounds(var_idx, new_bound)
    status, objective, _ = solve_as_lp(cclone)
    set_bounds(var_idx, original_bound)

    return status, objective


def apply_branch_history(c, branch_history):
    for b in branch_history:
        b_var_idx = b[0]
        b_type = b[1]
        b_val = b[2]

        if b_type == consts.LOWER_BOUND:
            c.variables.set_lower_bounds(b_var_idx, b_val)
        elif b_type == consts.UPPER_BOUND:
            c.variables.set_upper_bounds(b_var_idx, b_val)


def get_data(context):
    node_data = context.get_node_data()
    if node_data is None:
        node_data = {'branch_history': []}

    return node_data


def get_clone(context):
    cclone = CPX.Cplex(context.c)

    node_data = get_data(context)
    apply_branch_history(cclone, node_data['branch_history'])

    return cclone

def get_sb_scores(context, candidate_idxs):
    """Get strong branching scores"""
    cclone = get_clone(context)
    status, parent_objective, dual_values = solve_as_lp(cclone)

    print(status, dual_values)
    sb_scores = []
    if status == consts.LP_OPTIMAL or status == consts.LP_ABORT_IT_LIM:
        context.curr_node_dual_values = np.asarray(dual_values)
        for var_idx in candidate_idxs:
            upper_status, upper_objective = get_branch_solution(context, cclone, var_idx, consts.LOWER_BOUND)
            lower_status, lower_objective = get_branch_solution(context, cclone, var_idx, consts.UPPER_BOUND)
            
            # Infeasibility leads to higher score as it helps in pruning the tree
            if upper_status == consts.LP_INFEASIBLE:
                upper_objective = consts.INFEASIBILITY_SCORE                
                # context.num_infeasible_right[var_idx] += 1
            if lower_status == consts.LP_INFEASIBLE:
                lower_objective = consts.INFEASIBILITY_SCORE
                # context.num_infeasible_left[var_idx] += 1

            # Calculate deltas
            delta_upper = max(upper_objective - parent_objective, params.EPSILON)
            delta_lower = max(lower_objective - parent_objective, params.EPSILON)

            # Calculate sb score
            sb_score = delta_lower * delta_upper
            sb_scores.append(sb_score)
            
            # print(, upper_status, upper_objective, lower_status, lower_objective)
            print('\n---', 'Var id: ', var_idx, 'Delta lower: ', delta_lower, 'Delta upper: ', delta_upper,  'Sb score: ', sb_score)

    else:
        print("Root LP infeasible...")

    return sb_scores, cclone

In [123]:
class VariableSelectionCallback(CPX_CB.BranchCallback):
    def __call__(self):
        self.times_called += 1     
        scores = get_sb_scores(self, self.vidxs)
        
        # pc = self.get_pseudo_costs(self.vidxs)
        # for i, vid in enumerate(self.vidxs):
        print(scores)

In [124]:
c = CPX.Cplex('instance_1.lp')
set_params(c, test=True, seed=3)
vnames = c.variables.get_names()
vidxs = c.variables.get_indices(vnames)

vsel_cb = c.register_callback(VariableSelectionCallback)
vsel_cb.times_called = 0
vsel_cb.vnames = vnames
vsel_cb.vidxs = vidxs
vsel_cb.c = c

start = time.time()
c.solve()
stop = time.time()

print(stop-start, vsel_cb.times_called)
print(c.solution.get_objective_value())
c.write('instance_1.mps')

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_RandomSeed                              3
CPXPARAM_MIP_Cuts_Cliques                        -1
CPXPARAM_MIP_Cuts_Covers                         -1
CPXPARAM_MIP_Cuts_FlowCovers                     -1
CPXPARAM_MIP_Cuts_Implied                        -1
CPXPARAM_MIP_Cuts_GUBCovers                      -1
CPXPARAM_MIP_Cuts_Gomory                         -1
CPXPARAM_MIP_Cuts_PathCut                        -1
CPXPARAM_MIP_Cuts_MIRCut                         -1
CPXPARAM_MIP_Cuts_Disjunctive                    -1
CPXPARAM_MIP_Limits_EachCutLimit                 0
CPXPARAM_MIP_Strategy_Search                     1
CPXPARAM_MIP_Cuts_ZeroHalfCut                    -1
CPXPARAM_MIP_Cuts_MCFCut                         -1
CPXPARAM_MIP_Cuts_LiftProj                       -1
CPXPARAM_MIP_Cuts

In [113]:
gp.setParam('OutputFlag', 0)

m = gp.read('instance_1.mps')
mlp = m.relax()
mlp.optimize()
parent_obj = mlp.objVal

delta_dict = {}
all_vars = mlp.getVars()
for var in all_vars:
    _delta = []
    
    # Down
    var.UB = 0
    mlp.optimize()
    child_down = mlp.objVal
    _delta.append(child_down - parent_obj)
    var.UB = 1
    
    
    # Up
    var.LB = 1
    mlp.optimize()
    child_up = mlp.objVal
    _delta.append(child_up - parent_obj)
    var.LB = 0
    
    delta_dict[var.varName] = _delta

In [114]:
delta_dict

{'x1': [8.0, 19.0],
 'x2': [8.0, 20.0],
 'x3': [36.66666666666667, 8.0],
 'x4': [25.0, 8.0],
 'x5': [12.0, 8.0],
 'x6': [0.0, 81.0],
 'x7': [0.0, 90.0],
 'x8': [0.0, 32.0],
 'x9': [0.0, 70.0],
 'x10': [0.0, 82.66666666666669]}